### An example notebook that calculates average hotel ratings for cities in Sweden and displays it on interactive kepler.gl map

In [1]:
import pandas as pd
import datetime
from booking_scraper import bkscraper
from keplergl import KeplerGl
import math

### Read dataframe

In [6]:
df = pd.read_csv('se.csv')

In [7]:
df.head(10)

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Stockholm,59.3294,18.0686,Sweden,SE,Stockholm,primary,972647.0,972647.0
1,Gothenburg,57.6717,11.9810,Sweden,SE,Västra Götaland,admin,600473.0,600473.0
2,Malmö,55.5932,13.0214,Sweden,SE,Skåne,admin,321845.0,321845.0
3,Uppsala,59.8498,17.6389,Sweden,SE,Uppsala,admin,164535.0,164535.0
4,Uppsala,59.8601,17.6400,Sweden,SE,Stockholm,NaN,133117.0,127734.0
5,Västerås,59.6173,16.5422,Sweden,SE,Västmanland,admin,127799.0,127799.0
6,Örebro,59.2669,15.1965,Sweden,SE,Örebro,admin,125817.0,125817.0
7,Linköping,58.4094,15.6257,Sweden,SE,Östergötland,admin,114582.0,114582.0
8,Helsingborg,56.0424,12.7210,Sweden,SE,Skåne,minor,112496.0,112496.0
9,Jönköping,57.7713,14.1650,Sweden,SE,Jönköping,admin,89780.0,83202.0


### Calculate average hotel ratings

In [8]:
def calculate_average_hotel_rating(city):
    '''Calculates the average hotel rating given a city name in Sweden.
    Args:
        city: a city name in Sweden
    Returns:
        Average hotel rating if at least 5 hotels with scores is found, 0 otherwise
    '''
    
    today = datetime.datetime.now()
    start_date = today + datetime.timedelta(7)
    end_date = start_date + datetime.timedelta(7)
    result = bkscraper.get_result(city=city,  country='Sweden', datein=start_date, dateout=end_date, limit=-1, detail=False, is_verbose=False)
    scores = []
    for hotel in result:
        if hotel.get('score'):
            scores.append(float(hotel['score'].replace(',','.')))
    if len(scores) > 4:
        return sum(scores)/len(scores)
    return 0.0

In [9]:
df['average_hotel_rating'] = df.city.apply(lambda x: calculate_average_hotel_rating(x))


KeyboardInterrupt: 

In [ ]:
# Use exp in order to amplify the differences between average ratings 
df['exp_average_hotel_rating'] = df.average_hotel_rating.apply(lambda x : math.exp(x))

### FIlter the cities with no scores

In [ ]:
df = df[df.exp_average_hotel_rating > 1]

### Display the map

In [2]:
from map_config import config

In [ ]:
df.to_csv('sweden_hotel_ratings.csv', index=False)

In [3]:
df = pd.read_csv('sweden_hotel_ratings.csv')

In [4]:
df.head(5)

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper,average_hotel_rating,exp_average_hotel_rating
0,Stockholm,59.3294,18.0686,Sweden,SE,Stockholm,primary,972647.0,972647.0,8.124713,3376.897419
1,Gothenburg,57.6717,11.9810,Sweden,SE,Västra Götaland,admin,600473.0,600473.0,8.116129,3348.035491
2,Malmö,55.5932,13.0214,Sweden,SE,Skåne,admin,321845.0,321845.0,8.170000,3533.343964
3,Uppsala,59.8498,17.6389,Sweden,SE,Uppsala,admin,164535.0,164535.0,8.064000,3177.976651
4,Uppsala,59.8601,17.6400,Sweden,SE,Stockholm,NaN,133117.0,127734.0,8.064000,3177.976651


In [5]:
# Load an empty map
map_1 = KeplerGl(height=500, data={'sweden_hotel_rating_data': df})
map_1.config = config
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'lbkm4b', 'type': '…

### Save the map

In [5]:
map_1.save_to_html(data={'sweden_hotel_rating_data': df}, file_name='sweden_hotel_rating_map.html')


Map saved to sweden_hotel_rating_map.html!
